In [36]:
# Passo a passo
# Passo 1: Abrir o navegador
# Passo 2: Carregar a lista de produtos (livros)
# Passo 3: Para cada item da lista (para cada livro)
    # Passo 4: Pesquisar no gutenberg: https://www.gutenberg.org/
    # Passo 5: Se não encontrar:
        # Pesquisar no https://books.toscrape.com/
    # Passo 6: Registrar preço e link do livro na planilha

In [37]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# Passo a passo
# Passo 1: Abrir o navegador
# criar o navegador
navegador = webdriver.Chrome()

In [38]:
# Passo 2: Carregar a lista de produtos (livros)
import pandas as pd

df_produtos = pd.read_excel("Produtos.xlsx")
print(df_produtos)

                    nome            autor   categoria  preco  link
0           Frankenstein     Mary Shelley    Classics    NaN   NaN
1       Romeo and Juliet      Shakespeare    Classics    NaN   NaN
2       The Great Gatsby       Fitzgerald    Classics    NaN   NaN
3  Algorithms to Live By  Brian Christian  Nonfiction    NaN   NaN
4                Sapiens     Yuval Harari     History    NaN   NaN
5  Smarter Faster Better   Charles Duhigg  Nonfiction    NaN   NaN


In [39]:
# Passo 3: Para cada item da lista (para cada livro)
    # Passo 4: Pesquisar no gutenberg: https://www.gutenberg.org/
def pesquisar_gutenberg(nome, autor, navegador):
    lista_palavras_autor = autor.split(" ")
    navegador.get("https://www.gutenberg.org/") 
    # selecionar 1 elemento no selenium
    # id -> identificador unico
    # class -> costuma identificar elementos parecidos (usada no CSS)
    # texto -> pelo texto contido no elemento
    # xpath -> muito específico

    # para listas de elementos:
    # class 
    # tag name

    elemento = navegador.find_element("class name", "search-input")
    elemento.send_keys(nome)
    elemento.send_keys(Keys.ENTER)

    lista_resultados = navegador.find_elements("class name", "booklink")
    link = None
    for resultado in lista_resultados:
        texto = resultado.text
        if nome.lower() in texto.lower():
            if all(palavra in texto for palavra in lista_palavras_autor):  
                link = resultado.find_element("class name", "link").get_attribute("href")
                break
    if link:
        preco = 0
    else:
        preco = None
    return link, preco

def pesquisar_bookstocrape(nome, autor, navegador):
    navegador.get("https://books.toscrape.com/") 
    lista_categorias = navegador.find_element("class name", "nav-list")
    try:
        lista_categorias.find_element("link text", categoria).click()
    except:
        print("Nenhuma categoria selecionada")

    
    link = None
    preco = None
    encontrado = False
    while not encontrado: # passar entre as páginas da categoria
        lista_resultados = navegador.find_elements("class name", "product_pod")
        # olhar todos os resultados da página
        for resultado in lista_resultados:
            elemento_h3 = resultado.find_element("tag name", "h3")
            elemento_do_link = elemento_h3.find_element("tag name", "a")
            titulo = elemento_do_link.get_attribute("title")
            if nome.lower() in titulo.lower():
                encontrado = True
                # verificar se o autor é esse mesmo
                link = elemento_do_link.get_attribute("href")
                preco = resultado.find_element("class name", "price_color").text
                break
        # passar pra proxima página
        # tentar clicar no botão next
        try:
            navegador.find_element("link text", "next").click()
        # se não existir, eu vou interromper o while
        except:
            break
    return link, preco

for linha in df_produtos.index:
    nome = df_produtos.loc[linha, "nome"]
    autor = df_produtos.loc[linha, "autor"]
    categoria = df_produtos.loc[linha, "categoria"]

    link1, preco1 = pesquisar_gutenberg(nome, autor, navegador)
    print(nome, link1, preco1)
    if not link1:
        link2, preco2 = pesquisar_bookstocrape(nome, autor, navegador)
        print(nome, link2, preco2)
        df_produtos.loc[linha, "preco"] = preco2
        df_produtos.loc[linha, "link"] = link2
    else:
        df_produtos.loc[linha, "preco"] = preco1
        df_produtos.loc[linha, "link"] = link1

    # Passo 5: Se não encontrar:
        # Pesquisar no https://books.toscrape.com/
    # Passo 6: Registrar preço e link do livro na planilha

Frankenstein https://www.gutenberg.org/ebooks/84 0
Romeo and Juliet https://www.gutenberg.org/ebooks/1513 0
The Great Gatsby https://www.gutenberg.org/ebooks/64317 0
Algorithms to Live By None None
Algorithms to Live By https://books.toscrape.com/catalogue/algorithms-to-live-by-the-computer-science-of-human-decisions_880/index.html £30.81
Sapiens None None
Sapiens https://books.toscrape.com/catalogue/sapiens-a-brief-history-of-humankind_996/index.html £54.23
Smarter Faster Better None None
Smarter Faster Better https://books.toscrape.com/catalogue/smarter-faster-better-the-secrets-of-being-productive-in-life-and-business_543/index.html £38.89


In [ ]:
df_produtos.to_excel("ProdutosAtualizado.xlsx", index=False)
navegador.quit()